In [1]:
'''https://subscription.packtpub.com/book/undefined/9781784394516/4/ch04lvl1sec37/finite-differences-in-options-pricing'''

""" Shared attributes and functions of FD """
import numpy as np
import scipy.linalg as linalg

# Discretization
In the weekly report, S is discretized by n and t is discretized by m.

The discretization below is different, where S is discretized by M and t is discretized by N. In addition, here price $i = 1, 2, 3, \ldots, M-2, M-1$ and time $j = N-1, N-2, \ldots, 2, 1, 0$. Consequently, we use backward difference for time t, which is the same as forward difference in the weekly report. 

In summary:
- explicit method use the option price of the previous time step with difference asset price to predict the option price of the current time step.
- implicit method use the option price of the next time step and with difference asset price to predict the option price of the current time step.
- Crank-Nicolson method taking the average of both explict and implicit method.
- semi-implicit uses a different difference method.

# FiniteDifferences Class
- The price method is the public method used for calling the specific finite difference scheme implementation.
- assigns all the required parameters in the __init__ constructor method.
- _setup_boundary_conditions_: This method sets up the boundary conditions of the grid structure as a NumPy two-dimensional array.
- _setup_coefficients_: This method sets up the necessary coefficients used for traversing the grid structure.
- _traverse_grid_: This method iterates the grid structure backward in time, storing the calculated values toward the first column of the grid.
-  _interpolate_: Using the final calculated values on the first column of the grid, this method will interpolate these values to find the option price that closely infers the initial stock price, S0.

In [5]:
class FiniteDifferences(object):

    def __init__(self, S0, K, r, T, sigma, Smax, M, N,
                 is_call=True):
        self.S0 = S0
        self.K = K
        self.r = r
        self.T = T
        self.sigma = sigma
        self.Smax = Smax
        self.M, self.N = int(M), int(N)  # Ensure M&N are integers
        self.is_call = is_call

        self.dS = Smax / float(self.M)
        self.dt = T / float(self.N)
        self.i_values = np.arange(self.M)
        self.j_values = np.arange(self.N)
        self.grid = np.zeros(shape=(self.M+1, self.N+1))
        self.boundary_conds = np.linspace(0, Smax, self.M+1)

    def _setup_boundary_conditions_(self):
        pass

    def _setup_coefficients_(self):
        pass

    def _traverse_grid_(self):
        """  Iterate the grid backwards in time """
        pass

    def _interpolate_(self):
        """
        Use piecewise linear interpolation on the initial
        grid column to get the closest price at S0.
        """
        return np.interp(self.S0, 
                         self.boundary_conds,
                         self.grid[:, 0])
        
    def price(self):
        self._setup_boundary_conditions_()
        self._setup_coefficients_()
        self._traverse_grid_()
        return self._interpolate_()

# The explicit method
- the backward difference with respect to t
- the central difference with respect to S
- the second-order difference with respect to S

To avoid instability issues, we need to set a reasonable intervals of time t, where $0 < \Delta t < \frac{1}{\sigma^2(M-1) + \frac{1}{2}r}$.

In [6]:
""" Explicit method of Finite Differences """
# import numpy as np

# from FiniteDifferences import FiniteDifferences

class FDExplicitEu(FiniteDifferences):

    def _setup_boundary_conditions_(self):
        if self.is_call:
            self.grid[:, -1] = np.maximum(
                self.boundary_conds - self.K, 0)
            self.grid[-1, :-1] = (self.Smax - self.K) * \
                                 np.exp(-self.r *
                                        self.dt *
                                        (self.N-self.j_values))
        else:
            self.grid[:, -1] = \
                np.maximum(self.K-self.boundary_conds, 0)
            self.grid[0, :-1] = (self.K - self.Smax) * \
                               np.exp(-self.r *
                                      self.dt *
                                      (self.N-self.j_values))

    def _setup_coefficients_(self):
        self.a = 0.5 * self.dt * ((self.sigma**2) *
                              (self.i_values**2) -
                              self.r*self.i_values)
        self.b = 1 - self.dt * ((self.sigma**2) * 
                              (self.i_values**2) + 
                              self.r)
        self.c = 0.5 * self.dt * ((self.sigma**2) * 
                              (self.i_values**2) + 
                              self.r*self.i_values)

    def _traverse_grid_(self):
        for j in reversed(self.j_values):
            for i in range(self.M)[2:]:
                self.grid[i,j] = self.a[i] * self.grid[i-1,j+1] +\
                                 self.b[i] * self.grid[i,j+1] + \
                                 self.c[i] * self.grid[i+1,j+1] 


In [21]:
# from FDExplicitEu import FDExplicitEu
option = FDExplicitEu(50, 50, 0.1, 1.0, 0.4, 100, 200, 10000, True)
print(option.price())

9.909575362261545
[45.2418709  45.24232332 45.24277575 ... 49.99850002 49.99900001
 49.9995    ]
45.242323322769096
44.942756339081186
45.242323322769096
45.241870901797974
[ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5
  6.   6.5  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5
 13.  13.5 14.  14.5 15.  15.5 16.  16.5 17.  17.5 18.  18.5 19.  19.5
 20.  20.5 21.  21.5 22.  22.5 23.  23.5 24.  24.5 25.  25.5 26.  26.5
 27.  27.5 28.  28.5 29.  29.5 30.  30.5 31.  31.5 32

In [12]:
# instability problems when m and n are chosen improperly
option = FDExplicitEu(50, 50, 0.1, 1.0, 0.4, 100, 200, 10000, False)
print(option.price())

5.39954594518403
0.0
0.01427012248106949
0.0


In [9]:
# instability problems when m and n are chosen improperly
option = FDExplicitEu(50, 50, 0.1, 5.0/12.0, 0.4, 100, 100, 100, False)
print(option.price())

-1.6291077072251005e+53


In [14]:
option = FDExplicitEu(50, 50, 0.1, 5.0/12.0, 0.4, 100, 10, 1000, False)
print(option.price()[1][2])

[24.88109417 24.88564968 24.89020621 ... 29.99216694 29.99791667
 30.        ]


# The implicit method
The instability problem of the explicit method can be overcome using the forward difference with respect to time:
- the forward difference with respect to t.
- the central difference with respect to S.
- the second-order difference with respect to S.

In [10]:
"""
Price a European option by the implicit method 
of finite differences. 
"""

# from FDExplicitEu import FDExplicitEu


class FDImplicitEu(FDExplicitEu):

    def _setup_coefficients_(self):
        self.a = 0.5*(self.r*self.dt*self.i_values -
                      (self.sigma**2)*self.dt*(self.i_values**2))
        self.b = 1 + \
                 (self.sigma**2)*self.dt*(self.i_values**2) + \
                 self.r*self.dt
        self.c = -0.5*(self.r * self.dt*self.i_values +
                       (self.sigma**2)*self.dt*(self.i_values**2))
        self.coeffs = np.diag(self.a[2:self.M], -1) + \
                      np.diag(self.b[1:self.M]) + \
                      np.diag(self.c[1:self.M-1], 1)

    def _traverse_grid_(self):
        """ Solve using linear systems of equations """
        P, L, U = linalg.lu(self.coeffs)
        aux = np.zeros(self.M-1)

        for j in reversed(range(self.N)):
            aux[0] = np.dot(-self.a[1], self.grid[0, j])
            x1 = linalg.solve(L, self.grid[1:self.M, j+1] + aux)
            x2 = linalg.solve(U, x1)
            self.grid[1 : self.M, j] = x2

In [11]:
# from FDImplicitEu import FDImplicitEu
option = FDImplicitEu(50, 50, 0.1, 5./12., 0.4, 100, 100, 100, False)
# print(option.price())

option = FDImplicitEu(50, 50, 0.1, 5./12., 0.4, 100, 100, 1000, False)
# print(option.price())

4.065801939431454
4.071594188049893


# Crank-Nicolson method
- The Crank-Nicolson method converges much more quickly using a combination of the explicit and implicit methods, taking the average of both. 
- The Python implementation of the Crank-Nicolson method is given in the following FDCnEu class, which inherits from the FDExplicitEu class and overrides only the _setup_coefficients_ and _traverse_grid_ methods. Save this file as FDCnEu.py:

In [12]:
class FDCnEu(FDExplicitEu):

    def _setup_coefficients_(self):
        self.alpha = 0.25*self.dt*(
            (self.sigma**2)*(self.i_values**2) -
            self.r*self.i_values)
        self.beta = -self.dt*0.5*(
            (self.sigma**2)*(self.i_values**2) +
            self.r)
        self.gamma = 0.25*self.dt*(
            (self.sigma**2)*(self.i_values**2) +
            self.r*self.i_values)
        self.M1 = -np.diag(self.alpha[2:self.M], -1) + \
                  np.diag(1-self.beta[1:self.M]) - \
                  np.diag(self.gamma[1:self.M-1], 1)
        self.M2 = np.diag(self.alpha[2:self.M], -1) + \
                  np.diag(1+self.beta[1:self.M]) + \
                  np.diag(self.gamma[1:self.M-1], 1)

    def _traverse_grid_(self):
        """ Solve using linear systems of equations """
        P, L, U = linalg.lu(self.M1)

        for j in reversed(range(self.N)):
            x1 = linalg.solve(L,
                              np.dot(self.M2,
                                     self.grid[1:self.M, j+1]))
            x2 = linalg.solve(U, x1)
            self.grid[1:self.M, j] = x2

In [15]:
# from FDImplicitEu import FDImplicitEu
option = FDCnEu(50, 50, 0.1, 5./12., 0.4, 100, 100, 100, False)
print(option.price())


option = FDCnEu(50, 50, 0.1, 5./12., 0.4, 100, 100, 1000, False)
print(option.price())

4.072254507998114
4.072238354486828


# Semi-implicit method
- a backward difference approximation for the time derivative.
- forward difference approximation for the first order S derivative
- a symmetric central difference approximation for the second order S derivative

In [ ]:
class FDImplicitEu(FDExplicitEu):

    def _setup_coefficients_(self):
        self.a = 0.5*(self.r*self.dt*self.i_values -
                      (self.sigma**2)*self.dt*(self.i_values**2))
        self.b = 1 + \
                 (self.sigma**2)*self.dt*(self.i_values**2) + \
                 self.r*self.dt
        self.c = -0.5*(self.r * self.dt*self.i_values +
                       (self.sigma**2)*self.dt*(self.i_values**2))
        self.coeffs = np.diag(self.a[2:self.M], -1) + \
                      np.diag(self.b[1:self.M]) + \
                      np.diag(self.c[1:self.M-1], 1)

    def _traverse_grid_(self):
        """ Solve using linear systems of equations """
        P, L, U = linalg.lu(self.coeffs)
        aux = np.zeros(self.M-1)

        for j in reversed(range(self.N)):
            aux[0] = np.dot(-self.a[1], self.grid[0, j])
            x1 = linalg.solve(L, self.grid[1:self.M, j+1]+aux)
            x2 = linalg.solve(U, x1)
            self.grid[1:self.M, j] = x2